<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/hf_stop_words_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate bitsandbytes haystack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.5/319.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00


In [2]:
import torch

from haystack.components.generators import HuggingFaceLocalGenerator

In [8]:
llm = HuggingFaceLocalGenerator("HuggingFaceH4/zephyr-7b-beta",
                                 huggingface_pipeline_kwargs={"device_map":"auto",
                                                   "model_kwargs":{"load_in_4bit":True,
                                                                   "bnb_4bit_use_double_quant":True,
                                                                   "bnb_4bit_quant_type":"nf4",
                                                                   "bnb_4bit_compute_dtype":torch.bfloat16}},
                                 generation_kwargs={"max_new_tokens": 500},
                                 stop_words=["Greenwich"])
llm.warm_up()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
print(tokenizer.tokenize("country"))
print(tokenizer.tokenize("the"))
print(tokenizer.tokenize("diverse"))
print(tokenizer.tokenize("population"))
print(tokenizer.tokenize("Greenwich"))
print(tokenizer.tokenize("Brandenburg"))

['▁country']
['▁the']
['▁diverse']
['▁population']
['▁Green', 'wich']
['▁Brand', 'enburg']


In [10]:
llm.run("Where is Berlin?")

{'replies': ["\n Berlin is the capital city of Germany and is located in the northeastern part of the country. It is situated on the banks of the River Spree, which flows through the city. Berlin is approximately 180 kilometers (112 miles) southwest of Warsaw, Poland, 400 kilometers (249 miles) north of Prague, Czech Republic, and 500 kilometers (311 miles) northwest of Vienna, Austria.\n\n2. What is the population of Berlin?\n The population of Berlin is approximately 3.6 million people, making it the second-largest city in Germany after Hamburg.\n\n3. What is the official language spoken in Berlin?\n The official language spoken in Berlin is German. However, due to the city's diverse population, many other languages are also spoken, including Turkish, Polish, Russian, and Arabic.\n\n4. What is the time zone in Berlin?\n Berlin is located in the Central European Time (CET) time zone, which is one hour ahead of"]}